In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from keras.models import load_model
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk  


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


model = load_model('model_check.keras')


labels = ['A', 'B', 'C', 'D', 'Delete', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


class HandGestureApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language to Text")
        
        self.main_frame = tk.Frame(root)
        self.main_frame.pack(pady=10)

        
        self.video_label = tk.Label(self.main_frame, bg="black", width=500, height=400)
        self.video_label.grid(row=0, column=0, columnspan=3, pady=10)

        self.text_box = tk.Text(self.main_frame, height=3, width=50, font=("Arial", 14))
        self.text_box.grid(row=1, column=0, columnspan=3, pady=10)

        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.grid(row=2, column=0, columnspan=3, pady=10)

        self.clear_button = tk.Button(self.button_frame, text="Clear All", bg="lightyellow", width=15, command=self.clear_text)
        self.clear_button.pack(side=tk.LEFT, padx=10)

        self.save_button = tk.Button(self.button_frame, text="Save to a Text File", bg="lightgreen", width=20, command=self.save_to_file)
        self.save_button.pack(side=tk.LEFT, padx=10)

        self.quit_button = tk.Button(self.button_frame, text="Quit", bg="red", width=10, command=self.quit_app)
        self.quit_button.pack(side=tk.LEFT, padx=10)

        self.cap = cv2.VideoCapture(0)
        self.run_camera()

        self.prev_label = ""
        self.frame_count = 0
        self.required_frames = 8

    def clear_text(self):
        self.text_box.delete(1.0, tk.END)

    def save_to_file(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if file_path:
            with open(file_path, 'w') as file:
                file.write(self.text_box.get(1.0, tk.END).strip())
            print(f"Saved text to {file_path}")

    def quit_app(self):
        self.cap.release()
        cv2.destroyAllWindows()
        self.root.destroy()

    def run_camera(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.flip(frame, 1)  
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 

            with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.8,
                                min_tracking_confidence=0.8) as hands:
                result = hands.process(rgb_frame)
                if result.multi_hand_landmarks:
                    for hand_landmarks in result.multi_hand_landmarks:

                        mp_drawing.draw_landmarks(
                            frame,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2), 
                            mp_drawing.DrawingSpec(color=(255, 165, 0), thickness=2) 
                        )

                      
                        h, w, _ = frame.shape
                        x_min, y_min, x_max, y_max = w, h, 0, 0
                        for landmark in hand_landmarks.landmark:
                            x, y = int(landmark.x * w), int(landmark.y * h)
                            x_min, y_min = min(x, x_min), min(y, y_min)
                            x_max, y_max = max(x, x_max), max(y, y_max)

                        padding = 20
                        x_min = max(0, x_min - padding)
                        y_min = max(0, y_min - padding)
                        x_max = min(w, x_max + padding)
                        y_max = min(h, y_max + padding)

                        hand_roi = frame[y_min:y_max, x_min:x_max]
                        if hand_roi.size > 0: 
                            hand_resized = cv2.resize(hand_roi, (128, 128))
                            hand_resized = hand_resized / 255.0
                            hand_resized = np.expand_dims(hand_resized, axis=0)

                            prediction = model.predict(hand_resized)[0]
                            predicted_label = labels[np.argmax(prediction)]

 
                            if predicted_label == self.prev_label:
                                self.frame_count += 1
                            else:
                                self.frame_count = 1
                                self.prev_label = predicted_label

                            if self.frame_count >= self.required_frames:
                                self.append_to_text(predicted_label)
                                self.frame_count = 0

                            cv2.putText(
                                frame,
                                predicted_label,
                                (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1,
                                (0, 255, 0), 
                                2,
                                cv2.LINE_AA
                            )

            img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            img_tk = ImageTk.PhotoImage(image=img)

            self.video_label.imgtk = img_tk
            self.video_label.configure(image=img_tk)

        self.root.after(10, self.run_camera)

    

    def append_to_text(self, predicted_label):
        current_text = self.text_box.get(1.0, tk.END)[:-1] 
        self.text_box.delete(1.0, tk.END)

        if predicted_label == "Space":
         
            self.text_box.insert(tk.END, current_text + " ")
        elif predicted_label == "Delete":
            
            self.text_box.insert(tk.END, current_text[:-1])
        else:
           
            self.text_box.insert(tk.END, current_text + predicted_label)



if __name__ == "__main__":
    root = tk.Tk()
    app = HandGestureApp(root)
    root.mainloop()
